In [11]:
from datetime import datetime, timedelta
# from pydantic import UUID4
import uuid
import pandas as pd
from typing import Union, Dict, Any, List

class Config:
    SESSION_TIMEOUT = 3600

class SessionManager:
    def __init__(self):
        self.session_data: Dict[str, Dict[str, List[Dict[str, Any]]]] = {}
        self.session_info: Dict[str, Dict[str, Any]] = {}
    
    def create_session(self):
        id = uuid.uuid4() 
        session_id = f"test_gen_{id}"
        self.session_info[session_id] = {
            'created_at': datetime.now(),
            'last_activity': datetime.now(),
            'status': 'active'
        }
        self.session_data[session_id] = {}
        return session_id
    
    def store(self , session_id: str, key: str, data: Union[Dict[str, Any], pd.DataFrame]):
        if session_id not in self.session_data:
            False
        
        if isinstance(data, pd.DataFrame):
            data = data.to_dict(orient='records')
        
        if key not in self.session_data[session_id]:
            self.session_data[session_id][key] = []
        self.session_data[session_id][key].append(data)
        self.session_data[session_id]["last_activity"] = datetime.now()
        # self.session_data[session_id][key]= data
        return True
    
    def get(self, session_id, key = None):
        if session_id not in self.session_data:
            return None
        data = self.session_data[session_id] if key is None else self.session_data[session_id].get(key, [])
        if isinstance(data, list) and all(isinstance(d, list) for d in data) and all(isinstance(d[0], dict) for d in data if d):  # Check if list of DataFrame
            return [pd.DataFrame(d) for d in data]
        return data
    
    def end_session(self, session_id):
        if session_id in self.session_info:
            self.session_info[session_id]["status"] = "inactive"
            return True
        return False

    def clean_inactive_session(self):
        current_time = datetime.now()
        timeout_threshold = timedelta(seconds=Config.SESSION_TIMEOUT)
        expired_sessions = [id for id, info in self.session_info.items() if current_time - info['last_activity'] > timeout_threshold]
        inactive_sessions = [id for id in self.session_info if self.session_info[id]['status'] == 'inactive']
        
        for id in expired_sessions+ inactive_sessions:
            self.session_info.pop(id, None)
            self.session_data.pop(id, None)
            

    def get_active_session_count(self):
        return sum(1 for session in self.session_info.values() if session['status'] == 'active')

sess_mgr = SessionManager()
sess_id1 = sess_mgr.create_session()
print(sess_mgr.end_session(sess_id1))
sess_id2 = sess_mgr.create_session()
sess_mgr.session_info

True


{'test_gen_593e5ea1-a360-4667-a9f5-68b96a49ca87': {'created_at': datetime.datetime(2025, 3, 16, 13, 13, 46, 262239),
  'last_activity': datetime.datetime(2025, 3, 16, 13, 13, 46, 262239),
  'status': 'inactive'},
 'test_gen_aee62add-fba6-4085-aecd-b46d47219222': {'created_at': datetime.datetime(2025, 3, 16, 13, 13, 46, 262239),
  'last_activity': datetime.datetime(2025, 3, 16, 13, 13, 46, 262239),
  'status': 'active'}}

In [12]:
for k,v in sess_mgr.session_info.items():
    print(k)
    print(v)
    print(v['status'])

test_gen_593e5ea1-a360-4667-a9f5-68b96a49ca87
{'created_at': datetime.datetime(2025, 3, 16, 13, 13, 46, 262239), 'last_activity': datetime.datetime(2025, 3, 16, 13, 13, 46, 262239), 'status': 'inactive'}
inactive
test_gen_aee62add-fba6-4085-aecd-b46d47219222
{'created_at': datetime.datetime(2025, 3, 16, 13, 13, 46, 262239), 'last_activity': datetime.datetime(2025, 3, 16, 13, 13, 46, 262239), 'status': 'active'}
active


In [13]:
sess_mgr.clean_inactive_session()

In [14]:
sess_mgr.session_info.items()

dict_items([('test_gen_aee62add-fba6-4085-aecd-b46d47219222', {'created_at': datetime.datetime(2025, 3, 16, 13, 13, 46, 262239), 'last_activity': datetime.datetime(2025, 3, 16, 13, 13, 46, 262239), 'status': 'active'})])

In [15]:
sess_mgr.get_active_session_count()

1

In [16]:
# dict01 = {
#     'A':'a'
# }

# 'A' in dict01

In [17]:
# Lis01 = ['A', 'B', 'X']

# 'A' in Lis01

---
---
# With Data Frame

---
---

In [18]:
# sess_mgr = SessionManager()
sess_id = sess_id2 #sess_mgr.create_session()

df1 = pd.DataFrame({'Name': ['Alice', 'Bob'], 'Score': [85, 90]})

df2 = pd.DataFrame({
    'Name': ['Alice', 'Bob', 'Charlie', 'Diana', 'Ethan'],
    'Age': [24, 31, 18, 45, 22],
    'Department': ['Engineering', 'Marketing', 'Sales', 'HR', 'Engineering'],
    'Salary': [75000, 65000, 50000, 85000, 72000],
    'Start_Date': pd.to_datetime(['2021-03-15', '2019-11-01', '2023-01-10', '2015-07-22', '2022-05-30']),
    'Performance': [4.2, 3.8, 3.6, 4.7, 4.0],
    'Remote': [True, False, True, False, True]
})

sess_mgr.store(sess_id, "table", df1)
sess_mgr.store(sess_id, "table", df2)

sess_mgr.store(sess_id, "input_output", {"input": "Hello", "output": "Hi"})
sess_mgr.store(sess_id, "context_feedback", {"context": "first", "feedback": "good"})

True

In [21]:
sess_mgr.get(sess_id)

{'table': [[{'Name': 'Alice', 'Score': 85}, {'Name': 'Bob', 'Score': 90}],
  [{'Name': 'Alice',
    'Age': 24,
    'Department': 'Engineering',
    'Salary': 75000,
    'Start_Date': Timestamp('2021-03-15 00:00:00'),
    'Performance': 4.2,
    'Remote': True},
   {'Name': 'Bob',
    'Age': 31,
    'Department': 'Marketing',
    'Salary': 65000,
    'Start_Date': Timestamp('2019-11-01 00:00:00'),
    'Performance': 3.8,
    'Remote': False},
   {'Name': 'Charlie',
    'Age': 18,
    'Department': 'Sales',
    'Salary': 50000,
    'Start_Date': Timestamp('2023-01-10 00:00:00'),
    'Performance': 3.6,
    'Remote': True},
   {'Name': 'Diana',
    'Age': 45,
    'Department': 'HR',
    'Salary': 85000,
    'Start_Date': Timestamp('2015-07-22 00:00:00'),
    'Performance': 4.7,
    'Remote': False},
   {'Name': 'Ethan',
    'Age': 22,
    'Department': 'Engineering',
    'Salary': 72000,
    'Start_Date': Timestamp('2022-05-30 00:00:00'),
    'Performance': 4.0,
    'Remote': True}]],
 'l

In [ ]:
retrieved_df_list = sess_mgr.get(sess_id, "table")
for df in retrieved_df_list:
    print(df, df.info())

print(sess_mgr.get(sess_id, "input_output"))
print(sess_mgr.get(sess_id, "context_feedback"))

AttributeError: 'str' object has no attribute 'info'